<a href="https://colab.research.google.com/github/jewelry715/likelion/blob/main/abalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np #---1
import csv #---2 
np.random.seed(1541) #---3

In [ ]:
RND_MEAN = 0
RND_STD = 0.0030
learning_rate = 0.001
#클린코딩->코드를 어떻게하면 깔끔하고 예쁘게 작성?

In [ ]:
#메인함수 구축 과정
def main_exec(epoch_count=10,mb_size=10,report=1,train_rate=0.7): #몇번 학습?,미니배치 사이즈,몇번 출력?학습데이터 비율
  load_dataset() #데이터 불러오기
  init_model() #파라미터 초기화
  train_and_test(epoch_count, mb_size, report, train_rate) #학습과 테스트를 관장하는 기능능

In [ ]:
def load_dataset():
  with open('/content/drive/MyDrive/abalone.csv') as csvfile:
    csvreader=csv.reader(csvfile)
    next(csvreader,None)#건너 뛰어야할 변수 이름,첫번째 오는 값을 쓰지 않겠다
    rows=[]
    for row in csvreader:
      rows.append(row)

  #전역변수
  global data,input_cnt,output_cnt
  input_cnt,output_cnt=10,1
  #버퍼 
  data=np.zeros([len(rows),input_cnt+output_cnt])#독립변수의 크기,종속변수의 크기
  # print(data)
  # print(data.shape)
  #원핫벡터 처리
  for n,row in enumerate(rows):
    if row[0]=='I':data[n,0]=1
    if row[0]=='M':data[n,1]=1
    if row[0]=='F':data[n,2]=1
    data[n,3:]=row[1:]
  print(data)
load_dataset()

[[ 0.      1.      0.     ...  0.101   0.15   15.    ]
 [ 0.      1.      0.     ...  0.0485  0.07    7.    ]
 [ 0.      0.      1.     ...  0.1415  0.21    9.    ]
 ...
 [ 0.      1.      0.     ...  0.2875  0.308   9.    ]
 [ 0.      0.      1.     ...  0.261   0.296  10.    ]
 [ 0.      1.      0.     ...  0.3765  0.495  12.    ]]


In [ ]:
with open('/content/drive/MyDrive/abalone.csv') as csvfile:
  csvreader=csv.reader(csvfile)
  next(csvreader,None)#건너 뛰어야할 변수 이름
  rows=[]
  for row in csvreader:
    rows.append(row)
print(rows[0:2])

[['M', '0.455', '0.365', '0.095', '0.514', '0.2245', '0.101', '0.15', '15'], ['M', '0.35', '0.265', '0.09', '0.2255', '0.0995', '0.0485', '0.07', '7']]


In [ ]:
def init_model():
  global weight,bias,input_cnt,output_cnt
  weight=np.random.normal(RND_MEAN,RND_STD,[input_cnt,output_cnt])
  bias=np.zeros([output_cnt])
  print(weight)
  print(bias)
init_model()

[[-0.00213145]
 [-0.00229987]
 [ 0.00650987]
 [ 0.00321815]
 [ 0.00353119]
 [ 0.00382796]
 [-0.00220395]
 [-0.00287452]
 [ 0.00041733]
 [ 0.00046872]]
[0.]


In [ ]:
print('총 데이터의 수(행):',data.shape[0])
mb_size=100
step_count=int(data.shape[0]*0.7)//mb_size
print('데이터의 70%의 미니배치 스텝 수:',step_count)#학습데이터를 기준삼아서 70%만 사용

총 데이터의 수(행): 4177
데이터의 70%의 미니배치 스텝 수: 29


In [ ]:
#세번째 기능 구축하기-학습과 테스트를 관장하는 기능/학습에 대한 결과 표시세번째 기능 구축하기-학습과 테스트를 관장하는 기능/학습에 대한 결과 표시
def train_and_test(epoch_count,mb_size,report,train_rate):
  step_count=arrange_data(mb_size,train_rate)#반환하는 값은 미니배치가 몇 덩어리로 쪼개지는지?
  test_x,test_y=get_test_data()
  #2중 반복문?미니배치를 단위로 돌리고 있음->1100에폭
  for epoch in range(epoch_count):#에폭 첫번째 반복문
    losses,accs=[],[] #전체 미니배치(1epoch)의 결과인 손실과 정확도를 받는 리스트 정의
    for n in range(step_count):#1번의 에폭이 돌아가는 동안 미니배치 스텝카운트 반복 수행
      train_x,train_y= get_train_data(mb_size,n)#미니배치사이즈,스텝이 개별값을 받아 학습데이터의 독립,종속변수 변환
      loss,acc=run_train(train_x,train_y)
      losses.append(loss)
      accs.append(acc)

    if report>0 and (epoch+1)%report==0: #report 주기에 따른 테스트 결과 출력
      acc=run_test(test_x,test_y)
      print('Epoch {} : train-loss = {:5.3f},accuracy={:5.3f}/Test = {:5.3f}'.\
            format(epoch+1,np.mean(losses),np.mean(accs),acc))#5자리 확보하되 세자리수까지만만
  final_acc=run_test(test_x,test_y)#최종 테스트 결과 별도 출력
  print("\n 최종 테스트 결과 : final accuracy = {:5.3f}".format(final_acc))

In [ ]:
def arrange_data(mb_size,train_rate): #데이터를 섞어주는 기능->미니배치 출력
  global data,shuffle_map,test_begin_index #다른 함수에서 활용할 수 있게 함
  #데이터를 무작위 셔플
  shuffle_map=np.arange(data.shape[0])
  np.random.shuffle(shuffle_map)
  #미니배치 스텝 카운트 출력
  step_count=int(data.shape[0]*train_rate)//mb_size
  #경계선을 탐색하는 코드
  test_begin_index=step_count*mb_size
  return step_count

In [ ]:
def get_train_data(mb_size,n):
  global data,shuffle_map,test_begin_index,output_cnt
  if n==0:
    np.random.shuffle(shuffle_map[:test_begin_index])
  train_data=data[shuffle_map[mb_size*n:mb_size*(n+1)]]
  return train_data[:,:-output_cnt],train_data[:,-output_cnt:]

In [ ]:
def get_test_data(): #테스트 데이터 분할 과정 기능 구축(변수 구분 포함ef get_test_data(): #테스트 데이터 분할 과정 기능 구축(변수 구분 포함)
  global data,shuffle_map,test_begin_index,output_cnt
  test_data=data[shuffle_map[test_begin_index:]]
  return test_data[:,:-output_cnt],test_data[:,-output_cnt:]

In [ ]:
def run_train(x,y):
  output,aux_nn=forward_neuralnet(x)
  loss,aux_pp=forward_postproc(output,y)
  accuracy=eval_accuracy(output,y) #여기까지 순전파+평가 과정
  g_loss=1.0
  g_output=backprop_postproc(g_loss,aux_pp)
  backprop_neuralnet(g_output,aux_nn) #기울기를 가지고 가중치,바이어스 경신
  return loss,accuracy

In [ ]:
def run_test(x,y):
  output,_=forward_neuralnet(x) #_:필요없는 것을 리턴
  accuracy=eval_accuracy(output,y)
  return accuracy

In [ ]:
#신경망의 순전파 연산 과정
def forward_neuralnet(x):
  global weight,bias
  output=np.matmul(x,weight)+bias
  #두번째 반환값인 x는 aux_nn으로 반환처리되며,역전파를 수행할 때 사용할 예정
  return output,x

In [ ]:
#신경망의 순전파에 따른 mse 도출 과정
def forward_postproc(output,y):
  #편차->제곱->평균(손실)
  diff=output-y
  square=np.square(diff)
  loss=np.mean(square)
  return loss,diff #손실,편차 반환->편차는 역전파를 수행하는 데 있어 필요하기 때문(aux_pp)

In [ ]:
def backprop_neuralnet(g_output,x):
  global weight,bias
  g_output_w=x.transpose()
  g_w=np.matmul(g_output_w,g_output)
  g_b=np.sum(g_output,axis=0)
  weight-=learning_rate*g_w
  bias-=learning_rate*g_b

In [ ]:
def backprop_postproc(g_loss,diff):
  shape=diff.shape
  # print(shape) (10,1)
  g_loss=1.0
  g_loss_square=np.ones(shape)/np.prod(shape)
  g_square_diff=2*diff #편차 기울기로 구함
  g_diff_output=1
  #여기까지가 부분 기울기
  g_square=g_loss_square*g_loss
  g_diff=g_square_diff*g_square
  g_output=g_diff_output*g_diff #최종값ef backprop_postproc(diff):
  return g_output

In [ ]:
def eval_accuracy(output,y):
  mdiff=np.mean(np.abs((output-y)/y))
  return 1-mdiff

In [ ]:
#main_exec()

[[ 0.      1.      0.     ...  0.101   0.15   15.    ]
 [ 0.      1.      0.     ...  0.0485  0.07    7.    ]
 [ 0.      0.      1.     ...  0.1415  0.21    9.    ]
 ...
 [ 0.      1.      0.     ...  0.2875  0.308   9.    ]
 [ 0.      0.      1.     ...  0.261   0.296  10.    ]
 [ 0.      1.      0.     ...  0.3765  0.495  12.    ]]
[[-0.00259186]
 [ 0.00237584]
 [-0.00144708]
 [ 0.00177559]
 [-0.00235898]
 [ 0.00079248]
 [-0.00214648]
 [-0.00022529]
 [-0.00325013]
 [ 0.00022441]]
[0.]
Epoch 1 : train-loss = 36.805,accuracy=0.524/Test = 0.792
Epoch 2 : train-loss = 8.642,accuracy=0.820/Test = 0.814
Epoch 3 : train-loss = 7.582,accuracy=0.815/Test = 0.810
Epoch 4 : train-loss = 7.452,accuracy=0.813/Test = 0.807
Epoch 5 : train-loss = 7.363,accuracy=0.813/Test = 0.806
Epoch 6 : train-loss = 7.292,accuracy=0.811/Test = 0.806
Epoch 7 : train-loss = 7.231,accuracy=0.812/Test = 0.804
Epoch 8 : train-loss = 7.180,accuracy=0.811/Test = 0.804
Epoch 9 : train-loss = 7.132,accuracy=0.811/Test = 

In [ ]:
#learning_rate=0.01

In [ ]:
#main_exec(epoch_count=1000,mb_size=100,report=10,train_rate=0.7)

[[ 0.      1.      0.     ...  0.101   0.15   15.    ]
 [ 0.      1.      0.     ...  0.0485  0.07    7.    ]
 [ 0.      0.      1.     ...  0.1415  0.21    9.    ]
 ...
 [ 0.      1.      0.     ...  0.2875  0.308   9.    ]
 [ 0.      0.      1.     ...  0.261   0.296  10.    ]
 [ 0.      1.      0.     ...  0.3765  0.495  12.    ]]
[[-0.00079189]
 [-0.00184453]
 [ 0.0023149 ]
 [-0.00331474]
 [ 0.00306099]
 [ 0.00332318]
 [ 0.00043965]
 [ 0.00557458]
 [-0.00138164]
 [ 0.00190839]]
[0.]
Epoch 10 : train-loss = 7.066,accuracy=0.810/Test = 0.813
Epoch 20 : train-loss = 6.811,accuracy=0.809/Test = 0.815
Epoch 30 : train-loss = 6.678,accuracy=0.810/Test = 0.813
Epoch 40 : train-loss = 6.591,accuracy=0.810/Test = 0.815
Epoch 50 : train-loss = 6.513,accuracy=0.811/Test = 0.814
Epoch 60 : train-loss = 6.445,accuracy=0.811/Test = 0.817
Epoch 70 : train-loss = 6.380,accuracy=0.813/Test = 0.815
Epoch 80 : train-loss = 6.321,accuracy=0.814/Test = 0.817
Epoch 90 : train-loss = 6.260,accuracy=0.814

In [ ]:
# print(weight)
# print(bias)

[[  0.38374249]
 [  1.45150031]
 [  1.39463766]
 [  3.94126614]
 [  4.87029147]
 [  4.32545175]
 [  4.91104661]
 [-15.02700746]
 [ -2.64677785]
 [ 11.61808133]]
[3.23020198]


In [ ]:
# new_x = [0, 1, 0, 0.685,0.545, 0.18,1.42,0.674,0.392,0.5]
# output = forward_neuralnet(new_x)
# print(output[0] + 1.5)

[13.93134667]
